In [1]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import datetime
from datetime import datetime, timedelta
#import plotly.graph_objs as go
import matplotlib.pyplot as plt
import csv
import seaborn as sns
%matplotlib inline
sns.set()
from PIL import Image

# Zika dataset:

In [2]:
import sys
import glob

path = r'/media/juliane_oliveira/My Passport/Datalake/SINAN/Zika_pasta/Zika'
filenames = glob.glob(path + "/*.csv")
filenames

['/media/juliane_oliveira/My Passport/Datalake/SINAN/Zika_pasta/Zika/ZIKA2015.csv',
 '/media/juliane_oliveira/My Passport/Datalake/SINAN/Zika_pasta/Zika/ZIKA2016.csv',
 '/media/juliane_oliveira/My Passport/Datalake/SINAN/Zika_pasta/Zika/ZIKA2017.csv',
 '/media/juliane_oliveira/My Passport/Datalake/SINAN/Zika_pasta/Zika/ZIKA2018.csv']

In [3]:
lst_df = []
for f in filenames:
    print(f)
    df = pd.read_csv(f, low_memory = False, encoding="iso-8859-1")                    
    lst_df.append(df)

/media/juliane_oliveira/My Passport/Datalake/SINAN/Zika_pasta/Zika/ZIKA2015.csv
/media/juliane_oliveira/My Passport/Datalake/SINAN/Zika_pasta/Zika/ZIKA2016.csv
/media/juliane_oliveira/My Passport/Datalake/SINAN/Zika_pasta/Zika/ZIKA2017.csv
/media/juliane_oliveira/My Passport/Datalake/SINAN/Zika_pasta/Zika/ZIKA2018.csv


In [4]:
df = pd.concat(lst_df,sort=True)

In [5]:
df.columns

Index(['classi_fin', 'co_bainfc', 'co_usualt', 'co_usucad', 'codisinf',
       'comuninf', 'copaisinf', 'coufinf', 'criterio', 'cs_escol_n',
       'cs_flxret', 'cs_gestant', 'cs_mening', 'cs_raca', 'cs_sexo',
       'cs_suspeit', 'cs_zona', 'doenca_tra', 'dt_digita', 'dt_encerra',
       'dt_invest', 'dt_nasc', 'dt_notific', 'dt_obito', 'dt_sin_pri',
       'dt_transdm', 'dt_transrm', 'dt_transrs', 'dt_transse', 'dt_transsm',
       'dt_transus', 'evolucao', 'flxrecebi', 'fonetica_n', 'id_agravo',
       'id_bairro', 'id_cns_sus', 'id_distrit', 'id_geo1', 'id_geo2',
       'id_logrado', 'id_mn_resi', 'id_municip', 'id_ocupa_n', 'id_pais',
       'id_regiona', 'id_rg_resi', 'id_unidade', 'ident_micr', 'in_aids',
       'in_vincula', 'migrado_w', 'nduplic_n', 'nobaiinf', 'nu_ano', 'nu_cep',
       'nu_ddd_tel', 'nu_idade_n', 'nu_lote_h', 'nu_lote_v', 'nu_notific',
       'nu_numero', 'nu_telefon', 'sem_not', 'sem_pri', 'sg_uf', 'sg_uf_not',
       'soundex', 'tp_not', 'tp_sistema', 'tpa

## We will create year and month from  variable 'dt_notific' in order to use in our work. (see dengue_case_file for the problematic relating to the variable tp extract the week)

In [6]:
df['Mycol'] =  df['dt_notific'].apply(pd.to_datetime)

In [7]:
df['year'] = pd.DatetimeIndex(df['Mycol']).year
df['month'] = pd.DatetimeIndex(df['Mycol']).month

In [8]:
df.year.value_counts() # number of cases per year

2016.0    281464
2015.0     56389
2017.0     32684
2018.0     15643
Name: year, dtype: int64

In [9]:
df.classi_fin.value_counts()

1.0    172711
8.0    113487
2.0     89053
Name: classi_fin, dtype: int64

# Construct the dataset of Zika with the desired variables

In [10]:
df = df[(df.classi_fin == 1)|(df.classi_fin == 8)]

In [11]:
data = df.filter(['id_mn_resi','year','cs_sexo','nu_idade_n'])

## Process to clean age group and padronize following the age category given in population from IBGE

In [12]:
data.nu_idade_n.unique()

array([4024., 4029., 4026., 4036., 4042., 4049., 4023., 4021., 4002.,
       4020., 4031., 4004., 4016., 4032., 4043., 4027., 4044., 4067.,
       4030., 4009., 4041., 3010., 4012., 4013., 4051., 4047., 4037.,
       4034., 2011., 4018., 3011., 4040., 4055., 4005., 4045., 4064.,
       4006., 4014., 4025., 4035., 4050., 4028., 4070., 4052., 4066.,
       3002., 2000., 4015., 4007., 4053., 4022., 4008., 4039., 4056.,
       4048., 4038., 4062., 4033., 4003., 3001., 4046., 4001., 4017.,
       4078., 4063., 4011., 4019., 4059., 3008., 4058., 4010., 3005.,
       4057., 4061., 4065., 4054., 4068., 2013., 4073., 3009., 4071.,
       4074., 4072., 4060., 4069., 4077., 2005., 3007., 4085., 3006.,
       4096., 4082., 4079., 4076., 4084., 4081., 3004., 2002., 3003.,
       4094., 3024., 4089., 2020., 2025., 4083., 4075., 4086., 4091.,
       2007., 2003., 2004., 4090., 2012., 2015., 4080., 4088., 2006.,
       4095., 2008., 4102., 2001., 2027., 4087., 2009., 2010., 4093.,
       2023., 4092.,

In [24]:
vector =[]
for value in data.nu_idade_n.astype(str):
    if value == '1': value = 2001
    elif value == '2': value = 2001
    elif value == '3': value = 3001
    elif 4<= int(value) < 10: value = '400'+value
    elif len(value) == 2: value = '40' + value
    vector.extend([value])

In [25]:
#vector

In [26]:
data['nu_idade_n_cor'] = vector

In [27]:
data.columns

Index(['codmunres', 'year', 'cs_sexo', 'cs_raca', 'nu_idade_n',
       'nu_idade_n_cor'],
      dtype='object')

In [28]:
#data[data['nu_idade_n']==80]

## Categorize age group
### 0,  101,  202,  303,  404,  505,  606,  707,  808,  909, 1010,
###       1111, 1212, 1313, 1414, 1515, 1616, 1717, 1818, 1919, 2024, 2529,
###       3034, 3539, 4044, 4549, 5054, 5559, 6064, 6569, 7074, 7579, 8099],

In [13]:
fx = []
for value in data.nu_idade_n:
    if 1000 <= int(value) <4005: faixa = '4' 
    elif 4005 <=int(value)< 4010: faixa = '509'
    elif 4010 <=int(value)< 4015: faixa = '1014'
    elif 4015 <=int(value)< 4020: faixa = '1519'
    elif 4020 <=int(value)< 4025: faixa = '2024'
    elif 4025 <=int(value)< 4030: faixa = '2529'
    elif 4030 <=int(value)< 4035: faixa = '3034'
    elif 4035 <=int(value)< 4040: faixa = '3539' 
    elif 4040 <=int(value)< 4045: faixa = '4044'   
    elif 4045 <=int(value)< 4050: faixa = '4549'
    elif 4050 <=int(value)< 4055: faixa = '5054'
    elif 4055 <=int(value)< 4060: faixa = '5559'
    elif 4060 <=int(value)< 4065: faixa = '6064'
    elif 4065 <=int(value)< 4070: faixa = '6569'
    elif 4070 <=int(value)< 4075: faixa = '7074'
    elif 4075 <=int(value)< 4080: faixa = '7579'
    elif 4080 <=int(value)< 8100: faixa = '8099'
    fx.extend([faixa])             

In [14]:
data['faixa'] = fx
data

,id_mn_resi,year,cs_sexo,nu_idade_n,faixa
2,290980.0,2015.0,F,4024.0,2024
3,510340.0,2015.0,M,4029.0,2529
4,251480.0,2015.0,F,4026.0,2529
6,172110.0,2015.0,F,4036.0,3539
7,291360.0,2015.0,F,4042.0,4044
...,...,...,...,...,...
15933,330455.0,2018.0,F,4027.0,2529
15934,330455.0,2018.0,F,4035.0,3539
15939,251220.0,2018.0,M,4005.0,509
15940,320530.0,2018.0,M,4006.0,509


In [15]:
data.cs_sexo.value_counts()

F    191532
M     94315
I       350
f         1
Name: cs_sexo, dtype: int64

In [16]:
data['count'] = 1

In [17]:
data= data.rename(columns={"id_mn_resi": "codmunres"})

In [18]:
dados = data.groupby(['codmunres','year','cs_sexo','faixa'])['count'].sum().reset_index()

In [19]:
dados.cs_sexo = dados.cs_sexo.replace('F', 1)
dados.cs_sexo= dados.cs_sexo.replace('M', 2)

In [20]:
dados.head()

,codmunres,year,cs_sexo,faixa,count
0,110001.0,2016.0,1,1014,1
1,110001.0,2016.0,1,1519,1
2,110001.0,2016.0,1,2024,4
3,110001.0,2016.0,1,2529,5
4,110001.0,2016.0,1,3034,5


In [21]:
dados.groupby(['year'])['count'].sum()

year
2015.0     50119
2016.0    212602
2017.0     17152
2018.0      6316
Name: count, dtype: int64

In [22]:
df.year.value_counts()

2016.0    212610
2015.0     50119
2017.0     17153
2018.0      6316
Name: year, dtype: int64

# Population

In [23]:
#pop = pd.read_csv('G:\Datalake\POPULACAO\csv\POPBR12.csv')   #'G:\Datalake\POPULACAO\pop_race_sex_age.csv')

In [24]:
pop = pd.read_csv('/media/juliane_oliveira/My Passport/Datalake/POPULACAO/estimativas_pop_sex_age_muni_cidacs/pop_2000_2017_SFA.csv')
pop = pop.drop(columns=['Unnamed: 0'])

In [25]:
pop.columns

Index(['MUNIC_RES', 'ANO', 'SEXO', 'FXETARIA', 'POPULACAO'], dtype='object')

In [26]:
pop.FXETARIA.unique()

array([   0,  101,  202,  303,  404,  505,  606,  707,  808,  909, 1010,
       1111, 1212, 1313, 1414, 1515, 1616, 1717, 1818, 1919, 2024, 2529,
       3034, 3539, 4044, 4549, 5054, 5559, 6064, 6569, 7074, 7579, 8099])

In [27]:
pop = pop.replace({'FXETARIA': {0: 4, 101:4, 202:4,  303:4,  404:4,
                                505: 509, 606: 509,  707: 509,  808: 509,  909: 509,
                                1010: 1014, 1111: 1014, 1212: 1014, 1313: 1014, 1414: 1014,
                                1515: 1519, 1616: 1519, 1717: 1519, 1818: 1519, 1919: 1519}})

In [28]:
pop.head()

,MUNIC_RES,ANO,SEXO,FXETARIA,POPULACAO
0,110001,2000,1,4,118
1,110001,2000,1,4,110
2,110001,2000,1,4,127
3,110001,2000,1,4,149
4,110001,2000,1,4,140


# Creating tables to map the standard incidence ratio of dengue per year

## expected number of cases per year and cummulated

In [29]:
teste = dados

In [32]:
teste = teste[(2015 <= teste.year) & (teste.year <= 2018) & (teste.cs_sexo != 'f')&(teste.cs_sexo != 'I')]

In [33]:
teste.cs_sexo.unique()

array([1, 2], dtype=object)

In [34]:
teste.year.unique()

array([2016., 2017., 2018., 2015.])

In [35]:
teste['faixa'] = teste['faixa'].astype(int)

In [36]:
teste.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30841 entries, 0 to 30948
Data columns (total 5 columns):
codmunres    30841 non-null float64
year         30841 non-null float64
cs_sexo      30841 non-null object
faixa        30841 non-null int64
count        30841 non-null int64
dtypes: float64(2), int64(2), object(1)
memory usage: 1.4+ MB


In [37]:
sum_observed_gender_age_year = []
for year in [2015,2016,2017,2018]:
    df_year = teste[teste.year == year]
    table_year = pd.pivot_table(df_year, values='count', index=['codmunres'], columns=['cs_sexo','faixa'])
    table_year=table_year.fillna(0)
    resulted_dataframe = table_year.sum().reset_index()
    sum_observed_gender_age_year.append(resulted_dataframe)

In [41]:
sum_observed_gender_age_year[3]

,cs_sexo,faixa,0
0,1,4,254.0
1,1,509,154.0
2,1,1014,175.0
3,1,1519,389.0
4,1,2024,570.0
5,1,2529,527.0
6,1,3034,479.0
7,1,3539,419.0
8,1,4044,306.0
9,1,4549,271.0


In [68]:
pop.head()

,MUNIC_RES,ANO,SEXO,FXETARIA,POPULACAO
0,110001,2000,1,4,118
1,110001,2000,1,4,110
2,110001,2000,1,4,127
3,110001,2000,1,4,149
4,110001,2000,1,4,140


# Population 

In [42]:
pop = pop.astype(int)

In [43]:
teste_pop = pop.groupby(['MUNIC_RES','ANO','SEXO','FXETARIA'])['POPULACAO'].sum().reset_index()
teste_pop.head()

,MUNIC_RES,ANO,SEXO,FXETARIA,POPULACAO
0,110001,2000,1,4,1496
1,110001,2000,1,509,1677
2,110001,2000,1,1014,1570
3,110001,2000,1,1519,1522
4,110001,2000,1,2024,1292


In [44]:
sum_pop_gender_age_year = []
for year in [2015,2016,2017,2018]:
    df_year = teste_pop[teste_pop.ANO == year]
    table_year = pd.pivot_table(df_year, values='POPULACAO', index=['MUNIC_RES'], columns=['SEXO','FXETARIA'])
    table_year=table_year.fillna(0)
    resulted_dataframe = table_year.sum().reset_index()
    sum_pop_gender_age_year.append(resulted_dataframe)

In [43]:
sum_pop_gender_age_year[0]

,SEXO,FXETARIA,0
0,1,4,7016987
1,1,509,7624144
2,1,1014,8725413
3,1,1519,8558868
4,1,2024,8630227
5,1,2529,8460995
6,1,3034,7717657
7,1,3539,6766665
8,1,4044,6320570
9,1,4549,5692013


# calculating r_ j (for each year)

In [45]:
r_j = []
for df1,df2 in zip(sum_observed_gender_age_year,sum_pop_gender_age_year):
    #df2.rename(columns={0:'pop'}, inplace=True)
    risco_relativo = df1[0]/df2[0]
    r_j.append(risco_relativo)

In [78]:
r_j[0]

0     0.000305
1     0.000239
2     0.000225
3     0.000346
4     0.000418
5     0.000427
6     0.000423
7     0.000424
8     0.000387
9     0.000346
10    0.000312
11    0.000272
12    0.000216
13    0.000181
14    0.000148
15    0.000122
16    0.000098
17    0.000291
18    0.000205
19    0.000188
20    0.000178
21    0.000182
22    0.000176
23    0.000172
24    0.000170
25    0.000158
26    0.000134
27    0.000122
28    0.000097
29    0.000091
30    0.000086
31    0.000077
32    0.000074
33    0.000062
Name: 0, dtype: float64

# calculating E_i for each municipality

# 2015

In [46]:
filter_2015 = teste_pop[teste_pop.ANO == 2015]

In [47]:
P_2015 = pd.pivot_table(filter_2015, values='POPULACAO', index=['MUNIC_RES'], columns=['SEXO','FXETARIA'])
P_2015;

In [48]:
E_2015 =[]
for i in range(len(P_2015)):
    E_i = sum(np.array(P_2015.iloc[i])*np.array(r_j[0]))
    E_2015.append([P_2015.index.values[i],E_i])

In [49]:
Expected_2015 = pd.DataFrame(E_2015,
                 columns=['codmunres', 'expected_2015'])

In [50]:
Expected_2015.head()

,codmunres,expected_2015
0,110001,5.453759
1,110002,24.550267
2,110003,1.382330
3,110004,20.296820
4,110005,4.047035


# 2016

In [51]:
filter_2016 = teste_pop[teste_pop.ANO == 2016]

In [52]:
P_2016 = pd.pivot_table(filter_2016, values='POPULACAO', index=['MUNIC_RES'], columns=['SEXO','FXETARIA'])
P_2016;

In [53]:
E_2016 =[]
for i in range(len(P_2016)):
    E_i = sum(np.array(P_2016.iloc[i])*np.array(r_j[1]))
    E_2016.append([P_2016.index.values[i],E_i])

In [54]:
Expected_2016 = pd.DataFrame(E_2016,
                 columns=['codmunres', 'expected_2016'])

In [55]:
Expected_2016.head()

,codmunres,expected_2016
0,110001,22.257983
1,110002,102.733737
2,110003,5.618589
3,110004,85.113488
4,110005,16.743137


# 2017

In [56]:
filter_2017 = teste_pop[teste_pop.ANO == 2017]

In [57]:
P_2017 = pd.pivot_table(filter_2017, values='POPULACAO', index=['MUNIC_RES'], columns=['SEXO','FXETARIA'])
P_2017;

In [58]:
E_2017 =[]
for i in range(len(P_2017)):
    E_i = sum(np.array(P_2017.iloc[i])*np.array(r_j[2]))
    E_2017.append([P_2017.index.values[i],E_i])

In [59]:
Expected_2017 = pd.DataFrame(E_2017,
                 columns=['codmunres', 'expected_2017'])

In [60]:
Expected_2017.head()

,codmunres,expected_2017
0,110001,1.768999
1,110002,8.595448
2,110003,0.441388
3,110004,6.934121
4,110005,1.340230


# Observed number of cases per municipality and year

In [62]:
observed = dados.groupby(['codmunres','year'])['count'].sum().reset_index()
observed.head()

,codmunres,year,count
0,110001.0,2016.0,42
1,110002.0,2016.0,13
2,110002.0,2017.0,2
3,110004.0,2016.0,2
4,110004.0,2017.0,1


In [63]:
observed = observed.astype(int)
observed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3912 entries, 0 to 3911
Data columns (total 3 columns):
codmunres    3912 non-null int64
year         3912 non-null int64
count        3912 non-null int64
dtypes: int64(3)
memory usage: 91.8 KB


In [64]:
observed.describe()

,codmunres,year,count
count,3912.000000,3912.000000,3912.000000
mean,313930.637781,2016.384202,73.156697
std,103712.499228,0.903351,676.430549
min,110001.000000,2015.000000,1.000000
25%,250647.500000,2016.000000,1.000000
50%,292770.000000,2016.000000,3.000000
75%,352485.000000,2017.000000,18.000000
max,539904.000000,2018.000000,32711.000000


In [65]:
observed[observed.year == 2015]

,codmunres,year,count
12,110010,2015,2
20,110020,2015,2
26,110030,2015,1
73,130260,2015,37
87,140005,2015,1
...,...,...,...
3774,520870,2015,38
3879,521975,2015,2
3891,522140,2015,3
3896,522160,2015,1


In [66]:
observed = observed[(2015 <= observed.year) & (observed.year <= 2018)]

In [67]:
observed = pd.pivot_table(observed, values='count', index=['codmunres'], columns=['year'])

In [68]:
observed = observed.fillna(0) 
observed

year,2015,2016,2017,2018
codmunres,,,,
110001,0.0,42.0,0.0,0.0
110002,0.0,13.0,2.0,0.0
110004,0.0,2.0,1.0,0.0
110005,0.0,30.0,9.0,0.0
110006,0.0,3.0,0.0,0.0
...,...,...,...,...
522205,0.0,5.0,0.0,0.0
522220,0.0,1.0,15.0,0.0
522230,0.0,0.0,4.0,1.0


# calculate the standard incidence ratio

In [69]:
codigo_muni = pd.read_excel('/media/juliane_oliveira/My Passport/Datalake/POPULACAO/População/POP_python/DTB_BRASIL_MUNICIPIO.xls')

In [70]:
codigo_muni.head()

,UF,Nome_UF,Mesorregião Geográfica,Nome_Mesorregião,Microrregião Geográfica,Nome_Microrregião,Município,Código Município Completo,Nome_Município
0,11,Rondônia,2,Leste Rondoniense,6,Cacoal,15,1100015,Alta Floresta D'Oeste
1,11,Rondônia,2,Leste Rondoniense,6,Cacoal,379,1100379,Alto Alegre dos Parecis
2,11,Rondônia,2,Leste Rondoniense,3,Ariquemes,403,1100403,Alto Paraíso
3,11,Rondônia,2,Leste Rondoniense,5,Alvorada D'Oeste,346,1100346,Alvorada D'Oeste
4,11,Rondônia,2,Leste Rondoniense,3,Ariquemes,23,1100023,Ariquemes


In [71]:
codigo_muni.columns

Index(['UF', 'Nome_UF', 'Mesorregião Geográfica', 'Nome_Mesorregião',
       'Microrregião Geográfica', 'Nome_Microrregião', 'Município',
       'Código Município Completo', 'Nome_Município'],
      dtype='object')

In [72]:
codigo_muni.rename(columns={'Código Município Completo':'codmuni_full'}, 
                 inplace=True);

In [73]:
codigo_muni['codmuni'] = codigo_muni.codmuni_full.floordiv(10)

In [74]:
codigo_muni.head()

,UF,Nome_UF,Mesorregião Geográfica,Nome_Mesorregião,Microrregião Geográfica,Nome_Microrregião,Município,codmuni_full,Nome_Município,codmuni
0,11,Rondônia,2,Leste Rondoniense,6,Cacoal,15,1100015,Alta Floresta D'Oeste,110001
1,11,Rondônia,2,Leste Rondoniense,6,Cacoal,379,1100379,Alto Alegre dos Parecis,110037
2,11,Rondônia,2,Leste Rondoniense,3,Ariquemes,403,1100403,Alto Paraíso,110040
3,11,Rondônia,2,Leste Rondoniense,5,Alvorada D'Oeste,346,1100346,Alvorada D'Oeste,110034
4,11,Rondônia,2,Leste Rondoniense,3,Ariquemes,23,1100023,Ariquemes,110002


In [75]:
observed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2304 entries, 110001 to 539904
Data columns (total 4 columns):
2015    2304 non-null float64
2016    2304 non-null float64
2017    2304 non-null float64
2018    2304 non-null float64
dtypes: float64(4)
memory usage: 90.0 KB


In [76]:
final = codigo_muni.set_index('codmuni').join(observed) 

In [77]:
final

,UF,Nome_UF,Mesorregião Geográfica,Nome_Mesorregião,Microrregião Geográfica,Nome_Microrregião,Município,codmuni_full,Nome_Município,2015,2016,2017,2018
codmuni,,,,,,,,,,,,,
110001,11,Rondônia,2,Leste Rondoniense,6,Cacoal,15,1100015,Alta Floresta D'Oeste,0.0,42.0,0.0,0.0
110037,11,Rondônia,2,Leste Rondoniense,6,Cacoal,379,1100379,Alto Alegre dos Parecis,0.0,2.0,0.0,0.0
110040,11,Rondônia,2,Leste Rondoniense,3,Ariquemes,403,1100403,Alto Paraíso,NaN,NaN,NaN,NaN
110034,11,Rondônia,2,Leste Rondoniense,5,Alvorada D'Oeste,346,1100346,Alvorada D'Oeste,NaN,NaN,NaN,NaN
110002,11,Rondônia,2,Leste Rondoniense,3,Ariquemes,23,1100023,Ariquemes,0.0,13.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
522200,52,Goiás,5,Sul Goiano,16,Pires do Rio,22005,5222005,Vianópolis,NaN,NaN,NaN,NaN
522205,52,Goiás,5,Sul Goiano,15,Meia Ponte,22054,5222054,Vicentinópolis,0.0,5.0,0.0,0.0
522220,52,Goiás,4,Leste Goiano,12,Entorno de Brasília,22203,5222203,Vila Boa,0.0,1.0,15.0,0.0


In [78]:
final = final.join(Expected_2015.set_index('codmunres')).join(Expected_2016.set_index('codmunres')).join(Expected_2017.set_index('codmunres'))

In [79]:
final.columns

Index([                     'UF',                 'Nome_UF',
        'Mesorregião Geográfica',        'Nome_Mesorregião',
       'Microrregião Geográfica',       'Nome_Microrregião',
                     'Município',            'codmuni_full',
                'Nome_Município',                      2015,
                            2016,                      2017,
                            2018,           'expected_2015',
                 'expected_2016',           'expected_2017'],
      dtype='object')

In [80]:
final['SIR_2015'] = final[2015]/final['expected_2015']
final['SIR_2016'] = final[2016]/final['expected_2016']
final['SIR_2017'] = final[2017]/final['expected_2017']

In [81]:
final = final.fillna(0)

In [82]:
final.describe()

,UF,Mesorregião Geográfica,Microrregião Geográfica,Município,codmuni_full,2015,2016,2017,2018,expected_2015,expected_2016,expected_2017,SIR_2015,SIR_2016,SIR_2017
count,5570.000000,5570.000000,5570.000000,5570.000000,5.570000e+03,5570.000000,5570.000000,5570.000000,5570.000000,5570.000000,5570.000000,5570.000000,5570.000000,5570.000000,5570.000000
mean,32.377738,4.264991,17.633573,15816.982585,3.253591e+06,8.998025,38.168761,3.079354,1.133932,8.994255,38.113106,3.076302,0.790351,0.670496,0.640963
std,9.833862,3.206175,15.514306,15997.299780,9.849103e+05,120.576050,553.738048,46.637950,14.331028,53.846964,230.659617,18.423088,6.863849,3.098080,6.285768
min,11.000000,1.000000,1.000000,13.000000,1.100015e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,25.000000,2.000000,6.000000,4507.250000,2.512126e+06,0.000000,0.000000,0.000000,0.000000,1.228085,5.130505,0.402413,0.000000,0.000000,0.000000
50%,31.000000,3.000000,13.000000,10400.500000,3.146280e+06,0.000000,0.000000,0.000000,0.000000,2.656389,11.104839,0.890050,0.000000,0.000000,0.000000
75%,41.000000,6.000000,24.000000,20853.000000,4.119190e+06,0.000000,2.000000,0.000000,0.000000,5.959009,24.918633,2.011962,0.000000,0.085063,0.000000
max,53.000000,15.000000,66.000000,72202.000000,5.300108e+06,6502.000000,32711.000000,2685.000000,533.000000,2953.562791,12645.584817,1008.788208,286.673200,76.012980,300.221087


In [83]:
final

,UF,Nome_UF,Mesorregião Geográfica,Nome_Mesorregião,Microrregião Geográfica,Nome_Microrregião,Município,codmuni_full,Nome_Município,2015,2016,2017,2018,expected_2015,expected_2016,expected_2017,SIR_2015,SIR_2016,SIR_2017
codmuni,,,,,,,,,,,,,,,,,,,
110001,11,Rondônia,2,Leste Rondoniense,6,Cacoal,15,1100015,Alta Floresta D'Oeste,0.0,42.0,0.0,0.0,5.453759,22.257983,1.768999,0.000000,1.886963,0.000000
110037,11,Rondônia,2,Leste Rondoniense,6,Cacoal,379,1100379,Alto Alegre dos Parecis,0.0,2.0,0.0,0.0,3.096934,12.714577,1.044944,0.000000,0.157300,0.000000
110040,11,Rondônia,2,Leste Rondoniense,3,Ariquemes,403,1100403,Alto Paraíso,0.0,0.0,0.0,0.0,4.506350,18.873684,1.562522,0.000000,0.000000,0.000000
110034,11,Rondônia,2,Leste Rondoniense,5,Alvorada D'Oeste,346,1100346,Alvorada D'Oeste,0.0,0.0,0.0,0.0,3.812652,15.479946,1.225413,0.000000,0.000000,0.000000
110002,11,Rondônia,2,Leste Rondoniense,3,Ariquemes,23,1100023,Ariquemes,0.0,13.0,2.0,0.0,24.550267,102.733737,8.595448,0.000000,0.126541,0.232681
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
522200,52,Goiás,5,Sul Goiano,16,Pires do Rio,22005,5222005,Vianópolis,0.0,0.0,0.0,0.0,3.289693,13.877755,1.126238,0.000000,0.000000,0.000000
522205,52,Goiás,5,Sul Goiano,15,Meia Ponte,22054,5222054,Vicentinópolis,0.0,5.0,0.0,0.0,1.817285,7.664959,0.623400,0.000000,0.652319,0.000000
522220,52,Goiás,4,Leste Goiano,12,Entorno de Brasília,22203,5222203,Vila Boa,0.0,1.0,15.0,0.0,1.236661,5.162838,0.420366,0.000000,0.193692,35.683221


In [89]:
final[2017].sum()

17152.0

In [88]:
data.year.value_counts()

2016.0    212610
2015.0     50119
2017.0     17153
2018.0      6316
Name: year, dtype: int64

In [90]:
final.to_excel('standard_incidence_ratio_zika.xls')